In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
#files for project
regSsn = pd.read_csv('../data/MRegularSeasonDetailedResults.csv')
seeds = pd.read_csv('../data/MNCAATourneySeeds.csv')
conferences = pd.read_csv('../data/MTeamConferences.csv')
tournResults = pd.read_csv('../data/MNCAATourneyCompactResults.csv')
masseyRnkgs = pd.read_csv('../data/MMasseyOrdinals_thru_Season2023_Day128.csv')

In [4]:
df = regSsn
df.columns

Index(['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc',
       'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR',
       'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3',
       'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF'],
      dtype='object')

Feature Engineering: 
- getting seasonal stats (i.e. ppg, astpg, blkspg, etc.)
- seeds
- and creating features like offensive efficiency, def efficiency, etc.

Now calculate advanced metrics for teams:  
1. offensive efficiency - total points scored / total possessions * 100
2. defensive efficiency - total points allowed / total opponent possessions * 100 
3. effective field goal % (eFG%) - (FGM + .5*3PM)/FGA
4. True shooting % (TS%) - (total points) / (2*(FGA+.44*FTA))
5. Turnover % (TO%) - (TO / (FGA + .44*FTA + TO)) * 100
6. Assist % (ast%) - (AST/FGM) * 100

total possessions = FGA + .44(FTA) - offensive rebounds + turnovers


Did this already


In [5]:
df['WTeamPPG'] = df['WScore'] / 1  # Assuming 1 game played
df['LTeamPPG'] = df['LScore'] / 1  # Assuming 1 game played
df['WTeamAPG'] = df['WAst'] / 1  # Assuming 1 game played
df['LTeamAPG'] = df['LAst'] / 1  # Assuming 1 game played
df['WTeamORPG'] = df['WOR'] / 1  # Assuming 1 game played
df['LTeamORPG'] = df['LOR'] / 1  # Assuming 1 game played
df['WTeamDRPG'] = df['WDR'] / 1
df['LTeamDRPG'] = df['LDR'] / 1
df['WTeamFGA'] = df['WFGA'] / 1
df['LTeamFGA'] = df['LFGA'] / 1
df['WTeamFGM'] = df['WFGM'] / 1
df['LTeamFGM'] = df['LFGM'] / 1
df['WTeamStl'] = df['WStl'] / 1
df['LTeamStl'] = df['LStl'] / 1
df['WTeamBlk'] = df['WBlk'] / 1
df['LTeamBlk'] = df['LBlk'] / 1
df['WTeamTO'] = df['WTO'] / 1
df['LTeamTO'] = df['LTO'] / 1
df['WTeamFouls'] = df['WPF'] / 1
df['LTeamFouls'] = df['LPF'] / 1
df['WTeamFGA3'] = df['WFGA3'] / 1
df['LTeamFGA3'] = df['LFGA3'] / 1
df['WTeamFGM3'] = df['WFGM3'] / 1
df['LTeamFGM3'] = df['LFGM3'] / 1
df['WTeamFTA'] = df['WFTA'] / 1
df['LTeamFTA'] = df['LFTA'] / 1
df['WTeamFTM'] = df['WFTM'] / 1
df['LTeamFTM'] = df['LFTM'] / 1

# Group by 'Season' and 'WinningTeamID' for winning teams
winning_team_stats = df.groupby(['Season', 'WTeamID']).agg(
    {'WScore': 'sum', 'LScore':'sum','WAst': 'sum', 'WOR': 'sum','LOR': 'sum','WDR':'sum','WFGA':'sum','LFGA':'sum','WFGM':'sum',
     'WStl':'sum','WBlk':'sum','WTO':'sum','LTO':'sum','WPF':'sum','WFGA3':'sum','WFGM3':'sum','WFTA':'sum','LFTA':'sum','WFTM':'sum',
     'WTeamPPG': 'count'}).reset_index()

winning_team_stats.rename(columns={'WTeamID': 'TeamID', 'WScore': 'TotalPoints','LScore':'OpponentPoints', 'WAst': 'TotalAssists', 'WOR': 'OffReb', 'LOR': 'OppOffReb',
                                   'WDR':'DefReb','WFGA':'FGA','LFGA':'OppFGA','WFGM':'FGM','WStl':'Stl','WBlk':'Blk','WTO':'TO','LTO':'OppTO',
                                   'WPF':'Fouls','WFGA3':'FGA3','WFGM3':'FGM3','WFTA':'FTA','LFTA':'OppFTA','WFTM':'FTM',
                                   'WTeamPPG': 'GamesPlayed'}, inplace=True)

# Group by 'Season' and 'LosingTeamID' for winning teams
losing_team_stats = df.groupby(['Season', 'LTeamID']).agg(
    {'LScore': 'sum', 'WScore': 'sum','LAst': 'sum','LOR': 'sum','WOR': 'sum','LDR':'sum','LFGA':'sum','WFGA':'sum','LFGM':'sum','LStl':'sum','LBlk':'sum','LTO':'sum',
     'LPF':'sum','LFGA3':'sum','LFGM3':'sum','LFTA':'sum','LFTM':'sum','LTeamPPG': 'count'}).reset_index()

losing_team_stats.rename(columns={'LTeamID': 'TeamID', 'LScore': 'TotalPoints', 'WScore': 'OpponentPoints','LAst': 'TotalAssists', 'LOR': 'OffReb','WOR': 'OppOffReb',
                                  'LDR':'DefReb','LFGA':'FGA','WFGA':'OppFGA','LFGM':'FGM','LStl':'Stl','LBlk':'Blk','LTO':'TO','WTO':'OppTO',
                                  'LPF':'Fouls','LFGA3':'FGA3','LFGM3':'FGM3','LFTA':'FTA','WFTA':'OppFTA','LFTM':'FTM',
                                  'LTeamPPG': 'GamesPlayed'}, inplace=True)

#concat (stack) winning_team_stats with losing_team_stats
team_stats = pd.concat([winning_team_stats, losing_team_stats], axis=0)

#average the stats to get pointspergame, assistspergame, etc. 
average_stats = team_stats.groupby(['Season', 'TeamID']).agg(
    {'TotalPoints': 'sum', 'OpponentPoints':'sum', 'TotalAssists': 'sum', 'TotalAssists': 'sum','OffReb': 'sum','OppOffReb': 'sum','DefReb': 'sum','FGA': 'sum',
     'OppFGA': 'sum','FGM': 'sum','Stl':'sum','Blk':'sum','TO':'sum','OppTO':'sum','Fouls':'sum','FGA3':'sum','FGM3':'sum','FTA':'sum','OppFTA':'sum','FTM':'sum',
     'GamesPlayed': 'sum'}).reset_index()
average_stats['PPG'] = average_stats['TotalPoints'] / average_stats['GamesPlayed']
average_stats['APG'] = average_stats['TotalAssists'] / average_stats['GamesPlayed']
average_stats['ORPG'] = average_stats['OffReb'] / average_stats['GamesPlayed']
average_stats['DRPG'] = average_stats['DefReb'] / average_stats['GamesPlayed']
average_stats['FGAPG'] = average_stats['FGA'] / average_stats['GamesPlayed']
average_stats['FGMPG'] = average_stats['FGM'] / average_stats['GamesPlayed']
average_stats['FGA3PG'] = average_stats['FGA3'] / average_stats['GamesPlayed']
average_stats['FGM3PG'] = average_stats['FGM3'] / average_stats['GamesPlayed']
average_stats['FTAPG'] = average_stats['FTA'] / average_stats['GamesPlayed']
average_stats['FTMPG'] = average_stats['FTM'] / average_stats['GamesPlayed']
average_stats['StlPG'] = average_stats['Stl'] / average_stats['GamesPlayed']
average_stats['BlkPG'] = average_stats['Blk'] / average_stats['GamesPlayed']
average_stats['TOPG'] = average_stats['TO'] / average_stats['GamesPlayed']
average_stats['FlsPG'] = average_stats['Fouls'] / average_stats['GamesPlayed']
average_stats['TotalPossessions'] = average_stats['FGA']+(.44*average_stats['FTA'])-average_stats['OffReb']+average_stats['TO']  #total possessions
average_stats['offEff'] = (average_stats['TotalPoints']/average_stats['TotalPossessions'])*100 #offensive efficiency
average_stats['OppTotalPossessions'] = average_stats['OppFGA']+(.44*average_stats['OppFTA'])-average_stats['OppOffReb']+average_stats['OppTO']  #total possessions
average_stats['defEff'] = (average_stats['OpponentPoints']/average_stats['OppTotalPossessions'])*100 # defensive efficiency
average_stats['effFGPctg'] = (average_stats['FGM']+(.5*average_stats['FGM3']))/average_stats['FGA'] # effective field goal pctg
average_stats['trueShtgPctg'] = average_stats['TotalPoints'] / (2*(average_stats['FGA']+.44*average_stats['FTA'])) #true shooting percentage
average_stats['TOPctg'] = (average_stats['TO'] / (average_stats['FGA']+(.44*average_stats['OppFTA'])+average_stats['TO']))*100 #turnover percentage
average_stats['AsstPctg'] = (average_stats['TotalAssists']/average_stats['FGM'])*100



In [244]:
average_stats.head(2) 

,Season,TeamID,TotalPoints,OpponentPoints,TotalAssists,OffReb,OppOffReb,DefReb,FGA,OppFGA,...,TOPG,FlsPG,TotalPossessions,offEff,OppTotalPossessions,defEff,effFGPctg,trueShtgPctg,TOPctg,AsstPctg
0,2003,1102,1603,1596,364,117,269,471,1114,1188,...,11.428571,18.750000,1527.76,104.924857,1165.56,136.929888,0.579443,0.605015,21.184197,67.910448
1,2003,1103,2127,2110,411,264,325,538,1508,1539,...,12.629630,19.851852,1892.12,112.413589,1554.88,135.701790,0.534814,0.585912,17.205885,56.070941


In [176]:
average_stats['defEff'].describe()

count    7255.000000
mean      124.821248
std        15.386540
min        84.498351
25%       113.644749
50%       123.385987
75%       135.054846
max       185.286510
Name: defEff, dtype: float64

In [18]:
average_stats['offEff'].describe()

count    7255.000000
mean      102.604737
std         6.712376
min        74.769329
25%        98.197662
50%       102.785754
75%       107.163263
max       125.303429
Name: offEff, dtype: float64

In [19]:
average_stats.columns

Index(['Season', 'TeamID', 'TotalPoints', 'OpponentPoints', 'TotalAssists',
       'OffReb', 'OppOffReb', 'DefReb', 'FGA', 'OppFGA', 'FGM', 'Stl', 'Blk',
       'TO', 'OppTO', 'Fouls', 'FGA3', 'FGM3', 'FTA', 'OppFTA', 'FTM',
       'GamesPlayed', 'PPG', 'APG', 'ORPG', 'DRPG', 'FGAPG', 'FGMPG', 'FGA3PG',
       'FGM3PG', 'FTAPG', 'FTMPG', 'StlPG', 'BlkPG', 'TOPG', 'FlsPG',
       'TotalPossessions', 'offEff', 'OppTotalPossessions', 'defEff',
       'effFGPctg', 'trueShtgPctg', 'TOPctg', 'AsstPctg'],
      dtype='object')

In [20]:
print(len(average_stats))

7255


In [6]:
merged_df = average_stats.merge(seeds, on=['Season', 'TeamID'], how='left')


In [23]:
print(merged_df['Seed'].dtype)

object


In [7]:
def modify_seed1(entry):
    if pd.isna(entry):
        return '0'
    else:
        return entry[1:]

In [8]:
def modify_seed2(entry):
    if entry == '11a':
        return '11'
    if entry == '11b':
        return '11'
    if entry == '12b':
        return '12'
    if entry == '12a':
        return '12'
    if entry == '13b':
        return '13'
    if entry == '13a':
        return '13'
    if entry == '14a':
        return '14'
    if entry == '14b':
        return '14'
    if entry == '16a':
        return '16'
    if entry == '16b':
        return '16'
    else:
        return entry

In [9]:
merged_df['Seed'] = merged_df['Seed'].apply(modify_seed1)

In [10]:
merged_df['Seed'] = merged_df['Seed'].apply(modify_seed2)

In [243]:
merged_df.head(2)

,Season,TeamID,TotalPoints,OpponentPoints,TotalAssists,OffReb,OppOffReb,DefReb,FGA,OppFGA,...,FlsPG,TotalPossessions,offEff,OppTotalPossessions,defEff,effFGPctg,trueShtgPctg,TOPctg,AsstPctg,Seed
0,2003,1102,1603,1596,364,117,269,471,1114,1188,...,18.750000,1527.76,104.924857,1165.56,136.929888,0.579443,0.605015,21.184197,67.910448,0
1,2003,1103,2127,2110,411,264,325,538,1508,1539,...,19.851852,1892.12,112.413589,1554.88,135.701790,0.534814,0.585912,17.205885,56.070941,0


In [11]:
merged_df['Seed'] = merged_df['Seed'].astype(int)

In [12]:
print(sorted(merged_df['Seed'].unique()))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]


Massey Rankings: (thanks to https://www.kaggle.com/code/shaggy11/feature-engineering-flaml-submission)

In [13]:
massey = masseyRnkgs
massey.head()

,Season,RankingDayNum,SystemName,TeamID,OrdinalRank
0,2003,35,SEL,1102,159
1,2003,35,SEL,1103,229
2,2003,35,SEL,1104,12
3,2003,35,SEL,1105,314
4,2003,35,SEL,1106,260


In [14]:
list_seasons_unique = regSsn.Season.unique()
list_teams_season = massey.TeamID.unique()

In [15]:
dict_massey = {}
for season in massey.Season.unique():
    dict_massey[season] = {}
    massey_season = massey[massey["Season"]==season]
    list_teams_season = massey_season.TeamID.unique()
    ranking_days = massey_season.RankingDayNum
    q1_ranking_day_season = ranking_days.quantile(0.25)
    q3_ranking_day_season = ranking_days.quantile(0.75)
    for team in list_teams_season:
        massey_season_team = massey_season[massey_season["TeamID"]==team]
        dict_massey[season][team] = {}
        dict_massey[season][team]["Average Ranking"] = massey_season_team["OrdinalRank"].mean()
        dict_massey[season][team]["Early Ranking"] = massey_season_team[massey_season_team["RankingDayNum"]<q1_ranking_day_season]["OrdinalRank"].mean()
        dict_massey[season][team]["Median Ranking"] = massey_season_team[(massey_season_team["RankingDayNum"]>=q1_ranking_day_season) & (massey_season_team["RankingDayNum"]<=q3_ranking_day_season)]["OrdinalRank"].mean()
        dict_massey[season][team]["Late Ranking"] = massey_season_team[massey_season_team["RankingDayNum"]>q3_ranking_day_season]["OrdinalRank"].mean()
        dict_massey[season][team]["Ranking improvement Median - Early"] = - (dict_massey[season][team]["Median Ranking"] - dict_massey[season][team]["Early Ranking"])
        dict_massey[season][team]["Ranking improvement Late - Median"] =  - (dict_massey[season][team]["Late Ranking"] - dict_massey[season][team]["Median Ranking"])
        dict_massey[season][team]["Ranking improvement Late - Early"] =  - (dict_massey[season][team]["Late Ranking"] - dict_massey[season][team]["Early Ranking"])

In [36]:
dict_massey[2003][1102]

{'Average Ranking': 144.2875,
 'Early Ranking': 129.1829268292683,
 'Median Ranking': 143.847533632287,
 'Late Ranking': 158.3578947368421,
 'Ranking improvement Median - Early': -14.6646068030187,
 'Ranking improvement Late - Median': -14.510361104555102,
 'Ranking improvement Late - Early': -29.1749679075738}

In [16]:
df_massey_1 = []
columns = ["Season", "TeamID", "Average Ranking", "Early Ranking", "Median Ranking", "Late Ranking", "Ranking improvement Median - Early", "Ranking improvement Late - Median", "Ranking improvement Late - Early"]

i=0
for season in dict_massey.keys():
    for team in dict_massey[season].keys():
        if i==0:
            print(team)
            i+=1
        dict_season_team = dict_massey[season][team]
        dict_season_team["Season"]=season
        dict_season_team["TeamID"]=team
        df_massey_1.append(dict_season_team)
        
df_massey = pd.DataFrame(df_massey_1,columns=columns)

1102


In [189]:
df_massey.head()

,Season,TeamID,Average Ranking,Early Ranking,Median Ranking,Late Ranking,Ranking improvement Median - Early,Ranking improvement Late - Median,Ranking improvement Late - Early
0,2003,1102,144.287500,129.182927,143.847534,158.357895,-14.664607,-14.510361,-29.174968
1,2003,1103,182.205000,199.207317,184.264574,162.694737,14.942743,21.569837,36.512580
2,2003,1104,27.655502,17.233333,29.240343,33.642105,-12.007010,-4.401762,-16.408772
3,2003,1105,305.377500,291.268293,309.089686,308.842105,-17.821393,0.247581,-17.573813
4,2003,1106,243.265000,252.317073,233.520179,258.326316,18.796894,-24.806136,-6.009243


In [53]:
m=merged_df[(merged_df['Season']==2016) & (merged_df['TeamID']==1104)]

In [56]:
m['StlPG']

4431    5.625
Name: StlPG, dtype: float64

Late Season Record/winning percentage feature :

In [17]:
df_late_season = df[df['DayNum']>=100]
df_latelate_season = df[df['DayNum']>=115]

In [18]:
winning_games = df_late_season.groupby(['Season', 'WTeamID']).agg({'WScore':'count'}).reset_index()
winning_games.rename(columns={'WTeamID': 'TeamID','WScore':'GamesWon'}, inplace=True)

losing_games = df_late_season.groupby(['Season', 'LTeamID']).agg({'LScore':'count'}).reset_index()
losing_games.rename(columns={'LTeamID': 'TeamID','LScore':'GamesLost'}, inplace=True)


In [19]:
winning_games2 = df_latelate_season.groupby(['Season', 'WTeamID']).agg({'WScore':'count'}).reset_index()
winning_games2.rename(columns={'WTeamID': 'TeamID','WScore':'GamesWon'}, inplace=True)

losing_games2 = df_latelate_season.groupby(['Season', 'LTeamID']).agg({'LScore':'count'}).reset_index()
losing_games2.rename(columns={'LTeamID': 'TeamID','LScore':'GamesLost'}, inplace=True)


In [20]:
winning_games['GamesLost'] = losing_games['GamesLost']
winning_games2['GamesLost']= losing_games2['GamesLost']

In [113]:
winning_games.head()

,Season,TeamID,GamesWon,GamesLost
0,2003,1102,2,6
1,2003,1103,4,4
2,2003,1104,4,4
3,2003,1105,2,6
4,2003,1106,3,5


In [21]:
late_games = winning_games.copy()
late_games2 = winning_games2.copy()

In [22]:
late_games['Last32WinPctg'] = late_games['GamesWon']/(late_games['GamesWon']+late_games['GamesLost'])
late_games2['Last14WinPctg'] = late_games2['GamesWon']/(late_games2['GamesWon']+late_games2['GamesLost'])

In [165]:
late_games.head(2)

,Season,TeamID,GamesWon,GamesLost,Last32WinPctg
0,2003,1102,2,6,0.25
1,2003,1103,4,4,0.50


In [164]:
late_games2.head(2)

,Season,TeamID,GamesWon,GamesLost,Last14WinPctg
0,2003,1102,1,3,0.25
1,2003,1103,2,2,0.50


Preparing Tournament Data .. creating a target variable for the model.

In [23]:
late_games = late_games.drop(columns=['GamesWon','GamesLost'])
late_games2 = late_games2.drop(columns=['GamesWon','GamesLost'])

In [167]:
late_games.head()

,Season,TeamID,Last32WinPctg
0,2003,1102,0.250
1,2003,1103,0.500
2,2003,1104,0.500
3,2003,1105,0.250
4,2003,1106,0.375


Conference feature

In [197]:
conferences.head()

,Season,TeamID,ConfAbbrev
0,1985,1102,wac
1,1985,1103,ovc
2,1985,1104,sec
3,1985,1106,swac
4,1985,1108,swac


In [198]:
print(set(conferences['ConfAbbrev']))

{'mvc', 'summit', 'mac', 'aec', 'ecacn', 'pcaa', 'swac', 'big_twelve', 'cusa', 'gwc', 'pac_twelve', 'wcc', 'sec', 'ivy', 'big_south', 'caa', 'wac', 'mw_coll', 'mid_cont', 'big_west', 'a_sun', 'ecacm', 'ecacs', 'ovc', 'big_eight', 'mwc', 'a_ten', 'nec', 'meac', 'horizon', 'gmwc', 'big_east', 'asc', 'patriot', 'ecc', 'big_ten', 'nac', 'aac', 'pac_ten', 'big_sky', 'gsc', 'sun_belt', 'awc', 'acc', 'ind', 'swc', 'maac', 'metro', 'mw_city', 'southern', 'southland'}


In [24]:
power6Conferences = ['big_twelve','pac_twelve','big_east','big_ten','acc','sec']

In [25]:
conf = conferences.copy()
conf['Power6'] = 0
for index, row in conf.iterrows():
    conference = row['ConfAbbrev']
    
    if conference in power6Conferences:
        conf.at[index, 'Power6'] = 1

In [241]:
conf.head(2)

,Season,TeamID,ConfAbbrev,Power6
0,1985,1102,wac,0
1,1985,1103,ovc,0


Tourney Results Preparing

In [26]:
columns=["Season", "Team A", "Team B", "WinTeamA", "ScoreDiff"]
tournGames = []
for index, row in tournResults.iterrows():
    team_w_id = row["WTeamID"]
    team_l_id = row["LTeamID"]
    team_A = min(team_w_id, team_l_id)
    team_B = max(team_w_id, team_l_id)
    win_team_A = 1 if row["WTeamID"]==team_A else 0
    scoreDiff = row["WScore"] - row["LScore"]
    scoreDiff_team_A = scoreDiff if row["WTeamID"]==team_A else -scoreDiff
    dict_row = {"Season":row["Season"], "Team A": team_A, "Team B": team_B, "WinTeamA": win_team_A, "ScoreDiff": scoreDiff_team_A}
    tournGames.append(dict_row)

target_df = pd.DataFrame(tournGames, columns = columns)

In [242]:
target_df.head(2)

,Season,Team A,Team B,WinTeamA,ScoreDiff,PPG_A,APG_A,ORPG_A,DRPG_A,FGAPG_A,...,Ranking improvement Late - Median_B,Ranking improvement Late - Early_B,Last32WinPctg_A,Last32WinPctg_B,Last14WinPctg_A,Last14WinPctg_B,ConfAbbrev_A,Power6_A,ConfAbbrev_B,Power6_B
0,2003,1411,1421,0,-8,72.800000,14.200000,13.166667,24.800000,55.266667,...,-3.031154,-25.125160,0.454545,0.444444,0.625000,0.600000,swac,0,big_south,0
1,2003,1112,1436,1,29,85.214286,17.642857,15.178571,27.642857,65.714286,...,5.116592,19.243902,0.875000,0.700000,0.666667,0.666667,pac_ten,0,aec,0


In [203]:
print(len(target_df[target_df['WinTeamA']==1])/len(target_df))

0.5045317220543807


In [70]:
merged_df.columns

Index(['Season', 'TeamID', 'TotalPoints', 'OpponentPoints', 'TotalAssists',
       'OffReb', 'OppOffReb', 'DefReb', 'FGA', 'OppFGA', 'FGM', 'Stl', 'Blk',
       'TO', 'OppTO', 'Fouls', 'FGA3', 'FGM3', 'FTA', 'OppFTA', 'FTM',
       'GamesPlayed', 'PPG', 'APG', 'ORPG', 'DRPG', 'FGAPG', 'FGMPG', 'FGA3PG',
       'FGM3PG', 'FTAPG', 'FTMPG', 'StlPG', 'BlkPG', 'TOPG', 'FlsPG',
       'TotalPossessions', 'offEff', 'OppTotalPossessions', 'defEff',
       'effFGPctg', 'trueShtgPctg', 'TOPctg', 'AsstPctg', 'Seed'],
      dtype='object')

In [27]:
# avg stats like ppg, apg, etc.
columns = ['Season', 'TeamID','PPG', 'APG', 'ORPG', 'DRPG', 'FGAPG', 'FGMPG', 'FGA3PG',
       'FGM3PG', 'FTAPG', 'FTMPG', 'StlPG', 'BlkPG', 'TOPG', 'FlsPG']
avg_stats = merged_df[columns]

In [72]:
avg_stats.head(5)

,Season,TeamID,PPG,APG,ORPG,DRPG,FGAPG,FGMPG,FGA3PG,FGM3PG,FTAPG,FTMPG,StlPG,BlkPG,TOPG,FlsPG
0,2003,1102,57.250000,13.000000,4.178571,16.821429,39.785714,19.142857,20.821429,7.821429,17.107143,11.142857,5.964286,1.785714,11.428571,18.750000
1,2003,1103,78.777778,15.222222,9.777778,19.925926,55.851852,27.148148,16.074074,5.444444,25.851852,19.037037,7.259259,2.333333,12.629630,19.851852
2,2003,1104,69.285714,12.107143,13.571429,23.928571,57.178571,24.035714,19.857143,6.357143,20.928571,14.857143,6.607143,3.785714,13.285714,18.035714
3,2003,1105,71.769231,14.538462,13.500000,23.115385,61.615385,24.384615,20.769231,7.576923,21.846154,15.423077,9.307692,2.076923,18.653846,20.230769
4,2003,1106,63.607143,11.678571,12.285714,23.857143,55.285714,23.428571,17.642857,6.107143,16.464286,10.642857,8.357143,3.142857,17.035714,18.178571


In [28]:
# total stats like points in season, assists in season , etc
columns = ['Season', 'TeamID', 'TotalPoints', 'OpponentPoints', 'TotalAssists',
       'OffReb', 'OppOffReb', 'DefReb', 'FGA', 'OppFGA', 'FGM', 'Stl', 'Blk',
       'TO', 'OppTO', 'Fouls', 'FGA3', 'FGM3', 'FTA', 'OppFTA', 'FTM',
       'GamesPlayed']
tot_stats = merged_df[columns]

In [29]:
columns = ['Season', 'TeamID','Seed']
seeds = merged_df[columns]

In [30]:
# advanced stats like offensive efficiency , defensive efficiency, etc.
columns = ['Season', 'TeamID','offEff','defEff','effFGPctg', 'trueShtgPctg', 'TOPctg', 'AsstPctg',]
adv_stats = merged_df[columns]

In [31]:
target_df = target_df.copy()
target_df = target_df[target_df['Season']>=2003]
target_df = pd.merge(target_df,avg_stats,how="left", left_on=['Season','Team A'], right_on=['Season','TeamID']).drop(['TeamID'],axis=1)
target_df = pd.merge(target_df, avg_stats, how="left", left_on=["Season", "Team B"], right_on=["Season", "TeamID"], suffixes = ("_A", "_B")).drop(["TeamID"], axis=1)
target_df = pd.merge(target_df,tot_stats,how="left", left_on=['Season','Team A'], right_on=['Season','TeamID']).drop(['TeamID'],axis=1)
target_df = pd.merge(target_df, tot_stats, how="left", left_on=["Season", "Team B"], right_on=["Season", "TeamID"], suffixes = ("_A", "_B")).drop(["TeamID"], axis=1)
target_df = pd.merge(target_df,seeds,how="left", left_on=['Season','Team A'], right_on=['Season','TeamID']).drop(['TeamID'],axis=1)
target_df = pd.merge(target_df, seeds, how="left", left_on=["Season", "Team B"], right_on=["Season", "TeamID"], suffixes = ("_A", "_B")).drop(["TeamID"], axis=1)
target_df = pd.merge(target_df,adv_stats,how="left", left_on=['Season','Team A'], right_on=['Season','TeamID']).drop(['TeamID'],axis=1)
target_df = pd.merge(target_df, adv_stats, how="left", left_on=["Season", "Team B"], right_on=["Season", "TeamID"], suffixes = ("_A", "_B")).drop(["TeamID"], axis=1)
target_df = pd.merge(target_df,df_massey,how="left", left_on=['Season','Team A'], right_on=['Season','TeamID']).drop(['TeamID'],axis=1)
target_df = pd.merge(target_df, df_massey, how="left", left_on=["Season", "Team B"], right_on=["Season", "TeamID"], suffixes = ("_A", "_B")).drop(["TeamID"], axis=1)


In [236]:
target_df.head(5)

,Season,Team A,Team B,WinTeamA,ScoreDiff,PPG_A,APG_A,ORPG_A,DRPG_A,FGAPG_A,...,Ranking improvement Median - Early_A,Ranking improvement Late - Median_A,Ranking improvement Late - Early_A,Average Ranking_B,Early Ranking_B,Median Ranking_B,Late Ranking_B,Ranking improvement Median - Early_B,Ranking improvement Late - Median_B,Ranking improvement Late - Early_B
0,2003,1411,1421,0,-8,72.800000,14.200000,13.166667,24.800000,55.266667,...,28.859018,-26.041303,2.817715,241.370000,223.085366,245.179372,248.210526,-22.094006,-3.031154,-25.125160
1,2003,1112,1436,1,29,85.214286,17.642857,15.178571,27.642857,65.714286,...,5.666481,0.939020,6.605501,171.797500,184.243902,170.116592,165.000000,14.127311,5.116592,19.243902
2,2003,1113,1272,1,13,75.965517,15.551724,13.689655,23.310345,56.896552,...,35.353604,-1.431909,33.921694,42.000000,53.573171,46.035874,23.510000,7.537296,22.525874,30.063171
3,2003,1141,1166,1,6,79.344828,15.620690,10.586207,23.275862,52.689655,...,21.987149,21.353930,43.341078,16.869767,5.088889,18.255230,24.089109,-13.166341,-5.833879,-19.000220
4,2003,1143,1301,1,2,74.482759,16.000000,11.241379,24.379310,58.724138,...,46.589781,-0.314960,46.274821,55.451613,40.070588,61.968610,53.915789,-21.898022,8.052820,-13.845201


In [32]:
target_df = pd.merge(target_df,late_games,how="left", left_on=['Season','Team A'], right_on=['Season','TeamID']).drop(['TeamID'],axis=1)
target_df = pd.merge(target_df, late_games, how="left", left_on=["Season", "Team B"], right_on=["Season", "TeamID"], suffixes = ("_A", "_B")).drop(["TeamID"], axis=1)


In [33]:
target_df = pd.merge(target_df,late_games2,how="left", left_on=['Season','Team A'], right_on=['Season','TeamID']).drop(['TeamID'],axis=1)
target_df = pd.merge(target_df, late_games2, how="left", left_on=["Season", "Team B"], right_on=["Season", "TeamID"], suffixes = ("_A", "_B")).drop(["TeamID"], axis=1)

In [230]:
target_df.head(5)

,Team A,Team B,WinTeamA,ScoreDiff,PPG_A,APG_A,ORPG_A,DRPG_A,FGAPG_A,FGMPG_A,...,Median Ranking_B,Late Ranking_B,Ranking improvement Median - Early_B,Ranking improvement Late - Median_B,Ranking improvement Late - Early_B,Last32WinPctg_A,Last32WinPctg_B,Last14WinPctg_A,Season,Last14WinPctg_B


In [34]:
target_df = pd.merge(target_df,conf,how="left", left_on=['Season','Team A'], right_on=['Season','TeamID']).drop(['TeamID'],axis=1)
target_df = pd.merge(target_df, conf, how="left", left_on=["Season", "Team B"], right_on=["Season", "TeamID"], suffixes = ("_A", "_B")).drop(["TeamID"], axis=1)


In [240]:
target_df.head(5)

,Season,Team A,Team B,WinTeamA,ScoreDiff,PPG_A,APG_A,ORPG_A,DRPG_A,FGAPG_A,...,Ranking improvement Late - Median_B,Ranking improvement Late - Early_B,Last32WinPctg_A,Last32WinPctg_B,Last14WinPctg_A,Last14WinPctg_B,ConfAbbrev_A,Power6_A,ConfAbbrev_B,Power6_B
0,2003,1411,1421,0,-8,72.800000,14.200000,13.166667,24.800000,55.266667,...,-3.031154,-25.125160,0.454545,0.444444,0.625000,0.600000,swac,0,big_south,0
1,2003,1112,1436,1,29,85.214286,17.642857,15.178571,27.642857,65.714286,...,5.116592,19.243902,0.875000,0.700000,0.666667,0.666667,pac_ten,0,aec,0
2,2003,1113,1272,1,13,75.965517,15.551724,13.689655,23.310345,56.896552,...,22.525874,30.063171,0.500000,0.642857,0.666667,0.666667,pac_ten,0,cusa,0
3,2003,1141,1166,1,6,79.344828,15.620690,10.586207,23.275862,52.689655,...,-5.833879,-19.000220,0.900000,0.666667,0.857143,0.800000,mac,0,mvc,0
4,2003,1143,1301,1,2,74.482759,16.000000,11.241379,24.379310,58.724138,...,8.052820,-13.845201,0.555556,0.666667,0.400000,0.500000,pac_ten,0,acc,1


Now we have all the features we should need with everything in the right format.
Next step: EDA

for i in target_df.columns:
    print(i)

Features to get rid of:  
- ConfAbbrev_A and ConfAbbrev_B
- total Stats potentially

In [247]:
target_df = target_df.drop(columns=['ConfAbbrev_A','ConfAbbrev_B'])

Creating the difference between teamA and teamB stats:

In [249]:
list_feature_columns = list(set(["_".join(x.split("_")[:-1]) for x in list(target_df.columns)]))
list_feature_columns.remove("")
list_feature_columns

['trueShtgPctg',
 'TotalPoints',
 'Fouls',
 'Seed',
 'Last14WinPctg',
 'BlkPG',
 'OppFGA',
 'Early Ranking',
 'defEff',
 'OffReb',
 'GamesPlayed',
 'Late Ranking',
 'Ranking improvement Late - Early',
 'OppFTA',
 'FGMPG',
 'Median Ranking',
 'OppOffReb',
 'FGAPG',
 'OppTO',
 'OpponentPoints',
 'Ranking improvement Late - Median',
 'FTM',
 'Power6',
 'ORPG',
 'Average Ranking',
 'TotalAssists',
 'TOPG',
 'DefReb',
 'TOPctg',
 'APG',
 'effFGPctg',
 'FTMPG',
 'Last32WinPctg',
 'Blk',
 'StlPG',
 'FTAPG',
 'FGA',
 'PPG',
 'FGM3PG',
 'FGM3',
 'FTA',
 'FGA3PG',
 'Ranking improvement Median - Early',
 'TO',
 'offEff',
 'Stl',
 'FGM',
 'DRPG',
 'FGA3',
 'FlsPG',
 'AsstPctg']

In [250]:
def compute_A_minus_B(df_target, column):
    df_target[column+"_AmenoB"] = df_target[column+"_A"] - df_target[column+"_B"]
    return df_target

for col in list_feature_columns:
    target_df = compute_A_minus_B(target_df, col)

In [251]:
target_df.head()

,Season,Team A,Team B,WinTeamA,ScoreDiff,PPG_A,APG_A,ORPG_A,DRPG_A,FGAPG_A,...,FGA3PG_AmenoB,Ranking improvement Median - Early_AmenoB,TO_AmenoB,offEff_AmenoB,Stl_AmenoB,FGM_AmenoB,DRPG_AmenoB,FGA3_AmenoB,FlsPG_AmenoB,AsstPctg_AmenoB
0,2003,1411,1421,0,-8,72.800000,14.200000,13.166667,24.800000,55.266667,...,0.500000,50.953024,-13,2.653173,-12,35,1.627586,33,-0.803448,3.947052
1,2003,1112,1436,1,29,85.214286,17.642857,15.178571,27.642857,65.714286,...,4.588670,-8.460830,6,8.258702,38,129,1.918719,113,1.853448,0.963879
2,2003,1113,1272,1,13,75.965517,15.551724,13.689655,23.310345,56.896552,...,-7.482759,27.816308,6,3.754363,-63,27,-2.655172,-217,0.655172,-6.093630
3,2003,1141,1166,1,6,79.344828,15.620690,10.586207,23.275862,52.689655,...,-2.553814,35.153490,88,-4.162118,-71,-175,0.094044,-156,3.692790,0.072632
4,2003,1143,1301,1,2,74.482759,16.000000,11.241379,24.379310,58.724138,...,-5.465517,68.487803,-15,-2.338371,-43,63,2.345977,-181,-1.563218,-1.761993


In [253]:
train = target_df[target_df["Season"]<2016]

for col in target_df.columns:
    print(col)
    print(pd.isna(target_df[col]).sum())

Season
0
Team A
0
Team B
0
WinTeamA
0
ScoreDiff
0
PPG_A
0
APG_A
0
ORPG_A
0
DRPG_A
0
FGAPG_A
0
FGMPG_A
0
FGA3PG_A
0
FGM3PG_A
0
FTAPG_A
0
FTMPG_A
0
StlPG_A
0
BlkPG_A
0
TOPG_A
0
FlsPG_A
0
PPG_B
0
APG_B
0
ORPG_B
0
DRPG_B
0
FGAPG_B
0
FGMPG_B
0
FGA3PG_B
0
FGM3PG_B
0
FTAPG_B
0
FTMPG_B
0
StlPG_B
0
BlkPG_B
0
TOPG_B
0
FlsPG_B
0
TotalPoints_A
0
OpponentPoints_A
0
TotalAssists_A
0
OffReb_A
0
OppOffReb_A
0
DefReb_A
0
FGA_A
0
OppFGA_A
0
FGM_A
0
Stl_A
0
Blk_A
0
TO_A
0
OppTO_A
0
Fouls_A
0
FGA3_A
0
FGM3_A
0
FTA_A
0
OppFTA_A
0
FTM_A
0
GamesPlayed_A
0
TotalPoints_B
0
OpponentPoints_B
0
TotalAssists_B
0
OffReb_B
0
OppOffReb_B
0
DefReb_B
0
FGA_B
0
OppFGA_B
0
FGM_B
0
Stl_B
0
Blk_B
0
TO_B
0
OppTO_B
0
Fouls_B
0
FGA3_B
0
FGM3_B
0
FTA_B
0
OppFTA_B
0
FTM_B
0
GamesPlayed_B
0
Seed_A
0
Seed_B
0
offEff_A
0
defEff_A
0
effFGPctg_A
0
trueShtgPctg_A
0
TOPctg_A
0
AsstPctg_A
0
offEff_B
0
defEff_B
0
effFGPctg_B
0
trueShtgPctg_B
0
TOPctg_B
0
AsstPctg_B
0
Average Ranking_A
0
Early Ranking_A
0
Median Ranking_A
0
Late Ranking_

In [254]:
df.to_csv('target_df.csv', index=False)

In [255]:
target_df.to_csv('feat_games.csv', index=False)